this notebook is used to transform the 10240x10240 TIFs in 256x256 numpys per city each

In [11]:
import numpy  as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from osgeo import gdal
from PIL import Image
import rasterio
from rasterio import plot
from zipfile import ZipFile
import os
import random

In [12]:
def create_256_arrays_raster(raster_data):
    band_1 = raster_data.GetRasterBand(1).ReadAsArray()
    band_2 = raster_data.GetRasterBand(2).ReadAsArray()
    band_3 = raster_data.GetRasterBand(3).ReadAsArray()
    i = 0
    j = 0
    
    i_pixel = 0
    
    länge = band_1.shape[0]
    bilder = []
    overlap = 256
    while ((i_pixel+256) <= länge):
        j = 0
        j_pixel = 0
        höhe = band_1.shape[1]
        while ((j_pixel+256) <= höhe):
            bild1 = band_1[i_pixel:(i_pixel)+256, j_pixel:(j_pixel)+256]
            bild2 = band_2[i_pixel:(i_pixel)+256, j_pixel:(j_pixel)+256]
            bild3 = band_3[i_pixel:(i_pixel)+256, j_pixel:(j_pixel)+256]
            
            bild1 = bild1.reshape(256,256,1)
            bild2 = bild2.reshape(256,256,1)
            bild3 = bild3.reshape(256,256,1) 
            bild = np.concatenate((bild1, bild2), axis=2)
            bild = np.concatenate((bild, bild3), axis=2)
            
            bilder.append(bild)
            j = j+1
            j_pixel = j*overlap
        i = i+1
        i_pixel = i*overlap
        
    bilder = np.array(bilder)
    return bilder

In [13]:
def create_256_arrays_label(label_data):
    band_l = label_data.GetRasterBand(1).ReadAsArray()
    i = 0
    j = 0
    i_pixel = 0
    länge = band_l.shape[0]
    bilder = []
    overlap = 256
    while ((i_pixel+256) <= länge):
        j = 0
        j_pixel = 0
        höhe = band_l.shape[1]
        while ((j_pixel+256) <= höhe):
            bildl = band_l[i_pixel:(i_pixel)+256, j_pixel:(j_pixel)+256]
            bilder.append(bildl)
            j = j+1
            j_pixel = j*overlap
        i = i+1
        i_pixel = i*overlap
        
    bilder = np.array(bilder)
    return bilder

In [14]:
def file_length(file):
    file_length = 0
    start_path = file
    for path, dirs, files in os.walk(start_path):
        for f in files:
            fp = os.path.join(path, f)
            file_length += 1
            #total_size += os.path.getsize(fp)
    return file_length

In [16]:
def all_pictures_in_256_tiles_overlap(raster_file, label_file):
    flength = file_length(label_file)
    X = []
    y = []
    first = True
    for i in range(0,flength-1):
        raster_name = str(raster_file + '.' + str(i) + '.tif')
        label_name  = str( label_file + '.' + str(i) + '.tif') 
        
        raster_path = os.path.join(raster_file, raster_name)
        lable_path = os.path.join(label_file, label_name)
        
        raster_data = gdal.Open(raster_path)
        label_data = gdal.Open(lable_path)
        band_l = label_data.GetRasterBand(1).ReadAsArray()
        width, height = band_l.shape
        z = np.amax(band_l)
        if(width == 10240 and height == 10240):
                if(z != 0):
                    raster = create_256_arrays_raster(raster_data)
                    label =  create_256_arrays_label(label_data)
                    if first:
                        start = raster
                        startl = label
                        first = False
                    else:
                        start = np.concatenate((start, raster), axis=0)
                        startl = np.concatenate((startl, label), axis=0)
                    print("Raster " + str(i) + " durch")
    return start, startl

In [24]:
#X = ceate_256arrays_with_overlap_raster(raster_data)
#y = ceate_256arrays_with_overlap_label(label_data)
raster_file = '4.Stadt_10240'
label_file  = '4.StadtL_10240'

In [25]:
X, y = all_pictures_in_256_tiles_overlap(raster_file, label_file)

Raster 0 durch
Raster 2 durch
Raster 7 durch
Raster 8 durch
Raster 11 durch
Raster 12 durch
Raster 13 durch
Raster 15 durch
Raster 16 durch
Raster 17 durch
Raster 18 durch


In [ ]:
# def normalize(X,y): 
#     X = X / 255
#     length = y.shape[0]
#     y = y.reshape(length, 256, 256, 1)
#     return X,y
# X,y = normalize(X,y)

In [26]:
np.save('np/X_Stadt4', X)
np.save('np/y_Stadt4', y)